# Envio de mensagens via WhatsApp Web

Passos:
1) Criar a janela
2) Botão para selecionar arquivo de clientes
3) Label para mostrar caminho do arquivo de clientes
4) Label para mostrar quantidade total de mensagens a serem enviadas (contar quantos telefones E mensagens não vazias existem)
5) Botão para selecionar imagens
6) Label para mostrar caminho das imagens (apenas um unico string com todas as imagens da lista concatenadas)
7) Botão para iniciar processo de envio
8) Processo:  
    8.1) Apenas imprimir as mensagens  
    8.2) Label com dados atualizados na janela:  
        numero de enviados / numero total  
        percentual enviado 

Bibliotecas

In [4]:
# import libraries

# Basic Tkinter
import tkinter as tk
from tkinter import filedialog as fd
import tkinter.scrolledtext as st

# PIL to show images on Tkinter
from PIL import Image, ImageTk

# Pandas
import pandas as pd

# Just to get image file name from full path
from pathlib import Path

# Time to allow program wait few seconds during Chrome operations
import time

# To allow randomic waiting times (important to avoid Whatsapp account blocking)
import random

# Datetime to store current date of messages sent
from datetime import date

# necessary libraries for Chrome operations:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys


# pip install webdriver_manager
# This librari updates automatically the Browser Manger (in this case, Chrome)
from webdriver_manager.chrome import ChromeDriverManager

# Necessary to convert messages from ASCII text into URL aceptable addresses (convert special characters, spaces, etc)
import urllib


In [5]:
# global variables

# list of images to send
imgs_path = []

# current image on viewer
i = 0

Funções

In [6]:
def show_imgs():
    
    # Event Function to show the i image from list
    
    global tkphoto 
    # Canvas Size
    can_h = 400
    can_w = 400
    
    # esta demorando muito aqui, a solução de usar uma variavel global nao e interessante
    # https://blog.furas.pl/python-tkinter-why-label-doesnt-display-image-bug-with-garbage-collector-in-photoimage-GB.html
    # https://www.c-sharpcorner.com/blogs/basics-for-displaying-image-in-tkinter-python
    
    # Get the element i from list
    photo = Image.open(imgs_path[i])
    
    # Get the picture size (widht, height)
    pic_w, pic_h = photo.size

    # Calculate aspect image ratio
    aspect = pic_w/pic_h

    # if picture is wider than taller, the resizing limit will be the picture widht, limited to canvas width
    if aspect > 1:
        res_w = can_w
        res_h = can_w / aspect
    
    # else, the resizing limit will be the picture height, limited to canvas height
    else:
        res_h = can_h
        res_w = can_h * aspect
    

    # resize picture
    photo = photo.resize((int(res_w),int(res_h)))

    # create the Tkinter picture image object
    tkphoto = ImageTk.PhotoImage(photo) 

    # put the picture into the label
    lbl_photo = tk.Label(image=tkphoto,width=can_w,height=can_h,borderwidth=2,relief='solid')
    lbl_photo.grid(row=1,column=4,rowspan=4, padx=10,pady=10)
    
    # mostrar numero da imagem e o nome
    show_name()
    
    return()

In [7]:
def show_name():
    # função para mostrar numero de imagem, total de imagens e nome (sem o caminho)
    # .name extrai o nome do arquivo de um caminho completo
    img_name = '{} de {}: {}'.format(i+1,len(imgs_path),Path(imgs_path[i]).name)
    lbl_imgname = tk.Label(text=img_name,font=('Consolas 10'))
    lbl_imgname.grid(row=5, column=3,columnspan=3,sticky='NSEW',padx=10,pady=10)
    return()
    

In [8]:
def go_rgt():
    # função evento botão para selecionar imagem seguinte (direita)
    global i
    if i < len(imgs_path)-1:
        i = i + 1
    else:
        i = 0
    
    # chamar função para mostrar imagem
    show_imgs()

    return()

In [9]:
def go_lft():
    # função evento botão para selecionar imagem anterior (esquerda)
    global i
    if i > 0:
        i = i - 1
    else:
        i = len(imgs_path)-1
    
    # chamar função para mostrar imagem
    show_imgs()
    
    return()

In [10]:
def sel_imgs():
    # função evento selecionar lista de imagens
        
    global imgs_path
    global i
        
    # selecionar imagens
    imgs_path = sorted(list(fd.askopenfilenames(
        title='Selecione as imagens a enviar',
        filetypes=[
            ('Arquivos de imagen','.jpg'),
            ('Arquivos de imagen','.jpeg'),
            ('Arquivos de imagen','.png'),
            ('Arquivos de imagen','.gif'),]
    )))
    
    # Botão para esquerda
    btn_lft = tk.Button(text='<',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_lft)
    btn_lft.grid(row=1,column=3,rowspan=4,sticky='NSEW',padx=10,pady=10)

    # Botão para direita
    btn_rgt = tk.Button(text='>',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_rgt)
    btn_rgt.grid(row=1,column=5,rowspan=4,sticky='NSEW',padx=10,pady=10)
        
    # aqui chamar a função para mostrar a primeira imagem
    i = 0
    show_imgs()

    return()

In [11]:
def sel_file():
    
    # função evento selecionar o arquivo de clientes
    
    global contatos_df

    file_path = fd.askopenfilename(
        title='Selecione o arquivo Excel com a lista de destinatarios',
        filetypes=[
            ('Arquivo Excel','.xls'),
            ('Arquivo Excel','.xlsx')]
        )
    
    # ler arquivo Excel, precisa tres colunas
    contatos_df = pd.read_excel(file_path, sheet_name='CLIENTES')

    # remover registros com mensagens nulas
    contatos_df = contatos_df[~contatos_df['MENSAGEM'].isnull()]

    # resetar o indice
    contatos_df.reset_index(inplace=True)

    # eliminar nova coluna de indice criada
    contatos_df.drop(labels='index',axis=1,inplace=True)

    
    lbl_slctdfile['text'] = 'Serão enviadas {} mensagens.\n Clique em "Enviar Mensagens" para iniciar'.format(contatos_df['MENSAGEM'].count())
    
    return()

Processo Principal

In [12]:
def enviar_mensagens():
    
    # contar numero total de mensagens a enviar
    total = contatos_df['MENSAGEM'].count()

    # criar instancia do navegador Google Chrome
    navegador = webdriver.Chrome(ChromeDriverManager().install())
    
    # ir no site Whatsapp Web
    navegador.get("https://web.whatsapp.com/")
    time.sleep(5)

    # autorizar com o celular o QR Code
    # aguardar que carrege a pagina lateral com contatos
    while len(navegador.find_elements(By.ID,"side")) < 1:
        mainwindow.update()
        time.sleep(5)

        for j, mensagem in enumerate(contatos_df['MENSAGEM']):

            # this version considers all messages are not null
            # dataframe already cleaned up on opening file function

            # obter cliente e numero de telefone do dataframe
            nome = contatos_df.loc[j,"CLIENTE"]
            numero = contatos_df.loc[j, "TELEFONE"]
            
            # atualizar label de status
            lbl_enviando['text'] = 'Enviando mensagem {} de {}\nPara {} no telefone {}'.format(j+1,total,nome,numero)
            mainwindow.update()

            # transformar mensagem de ascii em url text
            texto = urllib.parse.quote(f"{mensagem}")

            # construir o link
            link = f"https://web.whatsapp.com/send?phone={numero}&text={texto}"

            # tentar enviar o link, se o numero der errado, pular para o proximo
            try:
                # colocar o link
                navegador.get(link)
                # mainwindow.update()
                # time.sleep(7)

                # aguardar aparecer lista de contatos, indicando que podemos apertar o enter
                while len(navegador.find_elements(By.ID,"side")) < 1:
                    # mainwindow.update()
                    time.sleep(2)
                    
                # enviar o ENTER
                navegador.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p/span').send_keys(Keys.ENTER)
                # mainwindow.update()
                # time.sleep(5)
                
                while len(navegador.find_elements(By.ID,"side")) < 1:
                    # mainwindow.update()
                    time.sleep(1)
                
                # para cada imagen da lista
                for imagen in imgs_path:
            
                    # find attachment button and click on it
                    navegador.find_element(By.CSS_SELECTOR,"span[data-icon='clip']").click()
                    time.sleep(2)

                    # select to find element and send all keys
                    navegador.find_element(By.CSS_SELECTOR,"input[type='file']").send_keys(imagen)
                    time.sleep(5)

                    # click on send attachment
                    navegador.find_element(By.CSS_SELECTOR,"span[data-icon='send']").click()
                    time.sleep(1)
                
                # escrever no text box resultado do envio
                txt_enviados.insert(tk.INSERT,'{}: {} recebeu a mensagem\n'.format(j+1,nome))
                
                # imprimir no terminal o resultado do envio
                print('{}:{}: recebeu a mensagem'.format(j+1,nome))
                # lista_enviados.append(date.today())

            except:
                    # escrever no text box resultado do envio
                    txt_enviados.insert(tk.INSERT,'{}: {} NÃO recebeu a mensagem\n'.format(j+1,nome))

                    # imprimir no terminal o resultado do envio
                    print('{}:{}: NÃO recebeu a mensagem'.format(j+1,nome))
                    # lista_enviados.append('Falha no Envio')
                    
            # aguardar tempo randomico entre 10 e 15 segundos
            # isto é importante para evitar ter o numero bloqueado pelo Whatsapp pelo uso de automações
            time.sleep(random.randint(5,10))            # nota: time.sleep não funciona no Tkinter, a menos que seja atualizada a janela
            
            mainwindow.update()

        lbl_slctdfile['text'] = 'Processo Finalizado'

            # aqui colocar os resultados de envio.
            # investigar como fazer somente leitura.
            # as configrações de state normal e disabled funcionam mas nao permitem alterar pelo programa        
            # txt_enviados.configure(state ='normal')
            # txt_enviados.insert(tk.INSERT,'{} recebeu a mensagem\n'.format(contatos_df['CLIENTE'][j]))
            # txt_enviados.configure(state ='disabled')
        
    return()

Design da janela principal

In [13]:
# criar a janela da aplicação
mainwindow = tk.Tk()

In [14]:
# configurações da janela
# mainwindow.geometry('750x500')
# mainwindow.rowconfigure(0,weight=1)
# mainwindow.columnconfigure([0,1],weight=1)

In [15]:
# Titulo da janela
mainwindow.title("Envio de Mensagens pelo WhatsApp")

''

In [16]:
# Label com titulo principal da janela
lbl_title = tk.Label(text="Envio de Mensagens pelo WhatsApp",font=('Consolas 15 bold underline'),borderwidth=1, relief='solid')
lbl_title.grid(row=0, column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [17]:
# Label de explicação
lbl_desc = tk.Label(text=
    """Este programa envia mensagens pelo WhatsApp Web,
    a partir de uma lista de destinatarios em formato Excel.
    A lista deve conter os campos
    Nome, Telefone e Mensagens, numa folha Clientes"""
    ,font=('Consolas 10'),wraplength=300,borderwidth=1, relief='solid')
lbl_desc.grid(row=1, column=0, columnspan=3,sticky='NSEW',padx=10,pady=10) 

In [18]:
# Label para seleção de arquivo Excel
lbl_file = tk.Label(text='Seleccione o arquivo Excel com os dados:',font=('Consolas 12'),anchor='e')
lbl_file.grid(row=3,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [19]:
# Botao para selecionar o arquivo em Excel
btn_file = tk.Button(text='Clique aqui para seleccionar o arquivo',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_file)
btn_file.grid(row=3,column=2,sticky='NSEW',padx=10,pady=10)

In [20]:
# Caminho e nome do arquivo seleccionado, posteriormente quantidade de mensagens
lbl_slctdfile = tk.Label(text='Nenhum Arquivo Seleccionado',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_slctdfile.grid(row=4,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [21]:
# Label para seleção dos arquivos de imagens
lbl_imgs = tk.Label(text='Seleccione as imagens a enviar:',font=('Consolas 12'),anchor='e')
lbl_imgs.grid(row=2,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [22]:
# Botão para selecionar imagens
btn_imgs = tk.Button(text='Clique aqui para seleccionar as imagens',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_imgs)
btn_imgs.grid(row=2,column=2,sticky='NSEW',padx=10,pady=10)

In [23]:
# Botão para rodar o processo
btn_test = tk.Button(text='Enviar Mensagens',font=('Consolas 10 bold'),command=enviar_mensagens)
btn_test.grid(row=5,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)
# pensa que enquanto esta rodando a janela, a informação dentro dela esta disponivel

In [24]:
# Label para colocar informações de estado de mensagen atual
lbl_enviando = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_enviando.grid(row=6,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [25]:
# Scroll text para mostrar a lista
# lbl_enviados = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
# lbl_enviados.grid(row=7,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)
txt_enviados = st.ScrolledText(mainwindow,width = 30, 
                            height = 8, 
                            font = ('Consolas 10'))
txt_enviados.grid(row=7,column = 0, columnspan=3,sticky='NSEW', pady = 10, padx = 10)

txt_enviados.insert(tk.INSERT,'')
# investigar como fazer read only


In [26]:
# janela principal
mainwindow.mainloop()